In [2]:
# determine peak flow
# Determine the Q(100) per "Asquith and Thompson"

In [7]:
import geopandas as gpd
from shapely.geometry import box
import os
import pandas as pd
import rasterio
from rasterio.transform import from_origin

In [4]:
# ---------------------------
# Function to extract pixel values from raster
def fn_extract_pixel_value(raster_file, x, y):
    with rasterio.open(raster_file) as src:
        # Convert x, y to row, col
        row, col = src.index(x, y)
        # Read pixel value
        value = src.read(1)[row, col]  # Assuming a single-band raster
        return value
# ---------------------------

# -----------------------
def fn_asquith_100yr_flow(flt_area, flt_precip, flt_slope_ft_per_mile):
    # https://pubs.usgs.gov/sir/2008/5084/pdf/sir2008-5084.pdf
    
    # This is a regional flow estimate for Texas (Q -100 year)
    # --- table 5 ----
    result_table5 = 10 ** 2.952 * flt_area ** 0.5186 * flt_precip ** 0.2366
    
    # --- table 4 ----
    result_table4 = 10 ** 0.3879 * flt_area ** 0.7133 * flt_precip ** 1.183 * flt_slope_ft_per_mile ** 0.6660
    return result_table4
# -----------------------

In [5]:
str_input_gpkg = r'E:\sample_2d_output\streams_mannings.gpkg'

# Load the GeoDataFrames
gdf_streams = gpd.read_file(str_input_gpkg)

In [ ]:
# convert gdf_streams containing multi-linestrings to linestrings.

In [22]:
# CONUS wide mean annual precipitation raster
str_mean_annual_precip_raster = r'E:\ras2fim-2d\input_gis\Mean_annual_precip_PRISM_5070.tif'

# Reproject gdf to match the CRS of the raster
gdf_streams_reproj = gdf_streams.to_crs(crs=rasterio.open(str_mean_annual_precip_raster).crs)

list_precip_inch = []
list_da_sq_mi = []
list_slope_per_mile = []

# Iterate over each line in GeoDataFrame
for idx, row in gdf_streams_reproj.iterrows():
    if row.geometry.geom_type == 'MultiLineString':
        # Iterate over each line in the MultiLineString
        for line in row.geometry.geoms:
            start_point = line.coords[0]
            start_x, start_y = start_point[0], start_point[1]
    else:
        # Extract starting point coordinates for single LineString
        start_point = row.geometry.coords[0]
        start_x, start_y = start_point[0], start_point[1]
    
    # ------- mean anual precip at starting point of that stream -------
    # Extract pixel value
    flt_precip_mm = fn_extract_pixel_value(str_mean_annual_precip_raster, start_x, start_y)
    
    # convert mm to inches
    flt_precip_inch = round(flt_precip_mm / 25.4, 1)
    
    list_precip_inch.append(flt_precip_inch)
    
    # ------- drainage area in square miles -------
    flt_da_sq_mi = round(row['tot_drainage_areasqkm'] * 0.386102,3)
    list_da_sq_mi.append(flt_da_sq_mi)
    
    
    # ------- assuming that slope of entire river is the same as the slope of that segment -------
    flt_slope_per_mile = round(row['So'] * 3.28084 * 1609.34, 2)
    list_slope_per_mile.append(flt_slope_per_mile)

In [29]:
# Create DataFrame
data = {
    'precip_in': list_precip_inch,
    'da_sq_mile': list_da_sq_mi,
    's_ft_per_mile': list_slope_per_mile
}

df = pd.DataFrame(data)

# Apply the function to each row and round the result to the nearest integer
df['flow_100_est'] = df.apply(lambda row: round(fn_asquith_100yr_flow(row['da_sq_mile'], row['precip_in'], row['s_ft_per_mile'])), axis=1)

In [30]:
df

,precip_in,da_sq_mile,s_ft_per_mile,flow_100_est
0,33.2,40.687,23.15,17548
1,33.1,115.306,13.06,25107
2,33.6,155.399,12.09,30034
3,33.6,198.389,12.70,36941
4,34.3,209.905,12.44,38869
...,...,...,...,...
67,35.3,246.475,4.33,22329
68,35.7,248.087,23.76,70647
69,32.2,1.507,50.27,2703
70,33.7,1.248,53.17,2588


In [ ]:
# I would like to get a ratio of this flow to limiting discharge (from csv)